In [42]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [11]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_mo.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,498553,498553,2660988802,4/13/21,768,PPP,EASTER SEALS MIDWEST,11933 Westline Industrial Dr,Saint Louis,MO,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,498554,498554,3162008008,6/24/20,768,PPP,"4M BUILDING SOLUTIONS, INC.",2827 Clark Avenue,SAINT LOUIS,MO,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,498555,498555,4904497002,4/4/20,709,PPP,KELLAN RESTAURANT MANAGEMENT CORP,1425 Swift Street Suite 200,KANSAS CITY,MO,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,498556,498556,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,4125 WILSON CREEK MARKETPLACE ROAD,BATTLEFIELD,MO,...,999 Other,5179 Other Telecommunications,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,NaN
4,498557,498557,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC","520 W. Pennway St., Ste 300",KANSAS CITY,MO,...,324 Petroleum & Coal Products,3241 Petroleum and Coal Products Manufacturing,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16345,514898,514898,9713238502,3/12/21,709,PPS,PREMIER PARKING SERVICES INC.,104 E 5th St Ste 204,Kansas City,MO,...,999 Other,8129 Other Personal Services,8129.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,2.916503e+14
16346,514899,514899,9761878500,3/12/21,709,PPS,J DOOLE PROMOTIONS INC,6130 NW Bell Rd,Parkville,MO,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,2.916503e+14
16347,514900,514900,9774858406,2/17/21,709,PPS,BUILDING CONTROL SOLUTIONS LLC,1420 NW Vivion Rd Ste 103,Kansas City,MO,...,999 Other,2382 Building Equipment Contractors,2382.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,NaN
16348,514901,514901,9851497109,4/15/20,709,PPP,"EPEC, INC.",5829 TROOST AVE,KANSAS CITY,MO,...,999 Other,8132 Grantmaking and Giving Services,8132.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,2.909501e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,2660988802,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354
1,3162008008,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490
2,4904497002,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370
3,5467867110,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495
4,5529817004,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [43]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MO_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,498553,498553,2660988802,4/13/21,768,PPP,EASTER SEALS MIDWEST,11933 Westline Industrial Dr,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,1,1,498554,498554,3162008008,6/24/20,768,PPP,"4M BUILDING SOLUTIONS, INC.",2827 Clark Avenue,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,2,2,498555,498555,4904497002,4/4/20,709,PPP,KELLAN RESTAURANT MANAGEMENT CORP,1425 Swift Street Suite 200,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,3,3,498556,498556,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,4125 WILSON CREEK MARKETPLACE ROAD,...,999 Other,5179 Other Telecommunications,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14
4,4,4,498557,498557,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC","520 W. Pennway St., Ste 300",...,324 Petroleum & Coal Products,3241 Petroleum and Coal Products Manufacturing,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14


In [44]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [45]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [46]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [47]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,2660988802,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,3162008008,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,4904497002,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,5467867110,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14
4,5529817004,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14
...,...,...,...,...,...
16345,9713238502,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,2.916503e+14
16346,9761878500,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,2.916503e+14
16347,9774858406,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,2.904702e+14
16348,9851497109,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,2.909501e+14


In [49]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() #5503

LoanNumber    35
full_add      35
Lat           35
Long          35
FIPS_z         0
dtype: int64

<h4>FIPS Script

In [25]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [26]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('29')] #MO
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


  0%|          | 0/3130 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [27]:
geo_id_list =geo_id 

Length Check

In [28]:
print(len(geo_id_list))

3130


Matching FIPS to DataFrame

In [29]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2684] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [30]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_23104/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
3,5467867110,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,[290770042021046]
4,5529817004,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,[290950153003025]
5,6105917109,"2001 ProEnergy Blvd, SEDALIA, MO",38.719938,-93.256005,[291594807001012]
13,8159007210,"2345 Grand Blvd., Suite 2400, KANSAS CITY, MO",39.085109,-94.580967,[290950158001067]
16,9274667002,"4321 Semple Avenue 0.0, St. Louis, MO",38.688229,-90.258626,[295101269003003]
...,...,...,...,...,...
16331,7794907410,"11301 Meadowside Drive, St. Louis, MO",38.691042,-90.417139,[291892150043011]
16333,7934748300,"111A Thunderboat Row, Camdenton, MO",38.008090,-92.744629,[290299505002000]
16335,8040887208,"1605 S. Big Bend Road #2207, St. Louis, MO",38.624706,-90.321079,[291892167003008]
16341,9274848603,"2925 E Battlefield St Ste 225, Springfield, MO",37.159071,-93.237387,[290770025021012]


Remove brackets

In [31]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_23104/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
3,5467867110,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,290770042021046
4,5529817004,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,290950153003025
5,6105917109,"2001 ProEnergy Blvd, SEDALIA, MO",38.719938,-93.256005,291594807001012
13,8159007210,"2345 Grand Blvd., Suite 2400, KANSAS CITY, MO",39.085109,-94.580967,290950158001067
16,9274667002,"4321 Semple Avenue 0.0, St. Louis, MO",38.688229,-90.258626,295101269003003
...,...,...,...,...,...
16331,7794907410,"11301 Meadowside Drive, St. Louis, MO",38.691042,-90.417139,291892150043011
16333,7934748300,"111A Thunderboat Row, Camdenton, MO",38.008090,-92.744629,290299505002000
16335,8040887208,"1605 S. Big Bend Road #2207, St. Louis, MO",38.624706,-90.321079,291892167003008
16341,9274848603,"2925 E Battlefield St Ste 225, Springfield, MO",37.159071,-93.237387,290770025021012


In [32]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
98,5313117007,"101 S Fifth St. Ste 200, COLUMBIA, MO",40.895540,-80.693380,NaN
172,5855317002,"16194 HIGHWAY 59, NEOSHO, MO",46.198482,-96.018793,NaN
364,6800117009,"6408 Highway 50, GERALD, MO",39.203595,-83.650158,NaN
1699,3975347104,"4780 N Service Road, SAINT PETERS, MO",44.649357,-83.939270,NaN
1840,1408137105,"340 KELLEY PKWY, MEXICO, MO",44.650646,-84.149050,NaN
3122,2598218607,"2650 County Highway 401 2650 County Hwy 401, B...",43.879441,-78.757768,NaN
3196,6533167007,"7700 East US 40 Highway, KANSAS CITY, MO",39.953602,-82.137167,NaN
3489,7904067104,"4760 N SERVICE RD, SAINT PETERS, MO",44.649357,-83.939470,NaN
4333,7509317009,"115 S 5th St, COLUMBIA, MO",40.895588,-80.693220,NaN
4913,9542958304,"1601 E Highway 84, Hayti, MO",31.522888,-90.580611,NaN


Parse for Merge

In [33]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
3,5467867110,290770042021046
4,5529817004,290950153003025
5,6105917109,291594807001012
13,8159007210,290950158001067
16,9274667002,295101269003003
...,...,...
16331,7794907410,291892150043011
16333,7934748300,290299505002000
16335,8040887208,291892167003008
16341,9274848603,290770025021012


Read in base data

In [34]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_mo.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MO_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,498553,498553,2660988802,4/13/21,768,PPP,EASTER SEALS MIDWEST,11933 Westline Industrial Dr,Saint Louis,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,1,498554,498554,3162008008,6/24/20,768,PPP,"4M BUILDING SOLUTIONS, INC.",2827 Clark Avenue,SAINT LOUIS,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,2,498555,498555,4904497002,4/4/20,709,PPP,KELLAN RESTAURANT MANAGEMENT CORP,1425 Swift Street Suite 200,KANSAS CITY,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,3,498556,498556,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,4125 WILSON CREEK MARKETPLACE ROAD,BATTLEFIELD,...,999 Other,5179 Other Telecommunications,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,NaN
4,4,498557,498557,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC","520 W. Pennway St., Ste 300",KANSAS CITY,...,324 Petroleum & Coal Products,3241 Petroleum and Coal Products Manufacturing,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,NaN


Merge with MainFrame

In [35]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,498553,498553,2660988802,4/13/21,768,PPP,EASTER SEALS MIDWEST,11933 Westline Industrial Dr,Saint Louis,...,6232 Residential Intellectual and Developmenta...,6232.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14,NaN
1,1,498554,498554,3162008008,6/24/20,768,PPP,"4M BUILDING SOLUTIONS, INC.",2827 Clark Avenue,SAINT LOUIS,...,5617 Services to Buildings and Dwellings,5617.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14,NaN
2,2,498555,498555,4904497002,4/4/20,709,PPP,KELLAN RESTAURANT MANAGEMENT CORP,1425 Swift Street Suite 200,KANSAS CITY,...,7225 Restaurants and Other Eating Places,7225.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14,NaN
3,3,498556,498556,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,4125 WILSON CREEK MARKETPLACE ROAD,BATTLEFIELD,...,5179 Other Telecommunications,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,NaN,290770042021046
4,4,498557,498557,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC","520 W. Pennway St., Ste 300",KANSAS CITY,...,3241 Petroleum and Coal Products Manufacturing,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,NaN,290950153003025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16345,16345,514898,514898,9713238502,3/12/21,709,PPS,PREMIER PARKING SERVICES INC.,104 E 5th St Ste 204,Kansas City,...,8129 Other Personal Services,8129.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,2.916503e+14,NaN
16346,16346,514899,514899,9761878500,3/12/21,709,PPS,J DOOLE PROMOTIONS INC,6130 NW Bell Rd,Parkville,...,3231 Printing and Related Support Activities,3231.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,2.916503e+14,NaN
16347,16347,514900,514900,9774858406,2/17/21,709,PPS,BUILDING CONTROL SOLUTIONS LLC,1420 NW Vivion Rd Ste 103,Kansas City,...,2382 Building Equipment Contractors,2382.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,NaN,290470202022009
16348,16348,514901,514901,9851497109,4/15/20,709,PPP,"EPEC, INC.",5829 TROOST AVE,KANSAS CITY,...,8132 Grantmaking and Giving Services,8132.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,2.909501e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [36]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,498553,498553,2660988802,4/13/21,768,PPP,EASTER SEALS MIDWEST,11933 Westline Industrial Dr,Saint Louis,...,6232.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14,NaN,291892150011052.0
1,1,498554,498554,3162008008,6/24/20,768,PPP,"4M BUILDING SOLUTIONS, INC.",2827 Clark Avenue,SAINT LOUIS,...,5617.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14,NaN,295101278002000.0
2,2,498555,498555,4904497002,4/4/20,709,PPP,KELLAN RESTAURANT MANAGEMENT CORP,1425 Swift Street Suite 200,KANSAS CITY,...,7225.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14,NaN,290470221006106.0
3,3,498556,498556,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,4125 WILSON CREEK MARKETPLACE ROAD,BATTLEFIELD,...,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,NaN,290770042021046,290770042021046
4,4,498557,498557,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC","520 W. Pennway St., Ste 300",KANSAS CITY,...,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,NaN,290950153003025,290950153003025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16345,16345,514898,514898,9713238502,3/12/21,709,PPS,PREMIER PARKING SERVICES INC.,104 E 5th St Ste 204,Kansas City,...,8129.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,2.916503e+14,NaN,291650306011020.0
16346,16346,514899,514899,9761878500,3/12/21,709,PPS,J DOOLE PROMOTIONS INC,6130 NW Bell Rd,Parkville,...,3231.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,2.916503e+14,NaN,291650303091000.0
16347,16347,514900,514900,9774858406,2/17/21,709,PPS,BUILDING CONTROL SOLUTIONS LLC,1420 NW Vivion Rd Ste 103,Kansas City,...,2382.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,NaN,290470202022009,290470202022009
16348,16348,514901,514901,9851497109,4/15/20,709,PPP,"EPEC, INC.",5829 TROOST AVE,KANSAS CITY,...,8132.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,2.909501e+14,NaN,290950081002015.0


Drop Extra FIPS Columns

In [37]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,498553,498553,2660988802,4/13/21,768,PPP,EASTER SEALS MIDWEST,11933 Westline Industrial Dr,Saint Louis,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,291892150011052.0
1,1,498554,498554,3162008008,6/24/20,768,PPP,"4M BUILDING SOLUTIONS, INC.",2827 Clark Avenue,SAINT LOUIS,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,295101278002000.0
2,2,498555,498555,4904497002,4/4/20,709,PPP,KELLAN RESTAURANT MANAGEMENT CORP,1425 Swift Street Suite 200,KANSAS CITY,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,290470221006106.0
3,3,498556,498556,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,4125 WILSON CREEK MARKETPLACE ROAD,BATTLEFIELD,...,999 Other,5179 Other Telecommunications,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,290770042021046
4,4,498557,498557,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC","520 W. Pennway St., Ste 300",KANSAS CITY,...,324 Petroleum & Coal Products,3241 Petroleum and Coal Products Manufacturing,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,290950153003025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16345,16345,514898,514898,9713238502,3/12/21,709,PPS,PREMIER PARKING SERVICES INC.,104 E 5th St Ste 204,Kansas City,...,999 Other,8129 Other Personal Services,8129.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,291650306011020.0
16346,16346,514899,514899,9761878500,3/12/21,709,PPS,J DOOLE PROMOTIONS INC,6130 NW Bell Rd,Parkville,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,291650303091000.0
16347,16347,514900,514900,9774858406,2/17/21,709,PPS,BUILDING CONTROL SOLUTIONS LLC,1420 NW Vivion Rd Ste 103,Kansas City,...,999 Other,2382 Building Equipment Contractors,2382.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,290470202022009
16348,16348,514901,514901,9851497109,4/15/20,709,PPP,"EPEC, INC.",5829 TROOST AVE,KANSAS CITY,...,999 Other,8132 Grantmaking and Giving Services,8132.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,290950081002015.0


In [38]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
98,98,498651,498651,5313117007,4/5/20,768,PPP,"FLAT BRANCH MORTGAGE, INC.",101 S Fifth St. Ste 200,COLUMBIA,...,999 Other,5223 Activities Related to Credit Intermediation,5223.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,250 to 499,"101 S Fifth St. Ste 200, COLUMBIA, MO",40.895540,-80.693380,NaN
172,172,498725,498725,5855317002,4/6/20,721,PPP,"OPAL FOODS COOPERATIVE, INC",16194 HIGHWAY 59,NEOSHO,...,999 Other,1123 Poultry and Egg Production,1123.0,SPRINGFIELD BRANCH OFFICE,d $2-5 million,250 to 499,"16194 HIGHWAY 59, NEOSHO, MO",46.198482,-96.018793,NaN
364,364,498917,498917,6800117009,4/7/20,768,PPP,"WOODY BOGLER TRUCKING COMPANY, WOODY BOGLER LO...",6408 Highway 50,GERALD,...,999 Other,4842 Specialized Freight Trucking,4842.0,ST. LOUIS DISTRICT OFFICE,d $2-5 million,100 to 249,"6408 Highway 50, GERALD, MO",39.203595,-83.650158,NaN
1699,1699,500252,500252,3975347104,4/12/20,768,PPP,"ED NAPLETON ST. LOUIS IMPORTS, INC",4780 N Service Road,SAINT PETERS,...,999 Other,4411 Automobile Dealers,4411.0,ST. LOUIS DISTRICT OFFICE,c $1-2 million,50 to 99,"4780 N Service Road, SAINT PETERS, MO",44.649357,-83.939270,NaN
1840,1840,500393,500393,1408137105,4/10/20,768,PPP,EAST CENTRAL MISSOURI BEHAVIORAL HEALTH SERVIC...,340 KELLEY PKWY,MEXICO,...,999 Other,6214 Outpatient Care Centers,6214.0,ST. LOUIS DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"340 KELLEY PKWY, MEXICO, MO",44.650646,-84.149050,NaN
3122,3122,501675,501675,2598218607,3/15/21,768,PPS,MIDWAY TRAILERS INC,2650 County Highway 401 2650 County Hwy 401,Benton,...,336 Transportation Equipment,3362 Motor Vehicle Body and Trailer Manufacturing,3362.0,ST. LOUIS DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2650 County Highway 401 2650 County Hwy 401, B...",43.879441,-78.757768,NaN
3196,3196,501749,501749,6533167007,4/7/20,709,PPP,"MISSOURI ORGANIC RECYCLING, INC.",7700 East US 40 Highway,KANSAS CITY,...,562 Hazardous Waste,5629 Remediation and Other Waste Management Se...,5629.0,KANSAS CITY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7700 East US 40 Highway, KANSAS CITY, MO",39.953602,-82.137167,NaN
3489,3489,502042,502042,7904067104,4/14/20,768,PPP,DAVE SINCLAIR LINCOLN MERCURY ST PETERS INC,4760 N SERVICE RD,SAINT PETERS,...,999 Other,4411 Automobile Dealers,4411.0,ST. LOUIS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"4760 N SERVICE RD, SAINT PETERS, MO",44.649357,-83.939470,NaN
4333,4333,502886,502886,7509317009,4/7/20,768,PPP,"FLATBRANCH BREWING, INC.",115 S 5th St,COLUMBIA,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,ST. LOUIS DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"115 S 5th St, COLUMBIA, MO",40.895588,-80.693220,NaN
4913,4913,503466,503466,9542958304,1/30/21,768,PPS,MID-CONTINENT AIRCRAFT CORPORATION,1601 E Highway 84,Hayti,...,999 Other,1151 Support Activities for Crop Production,1151.0,ST. LOUIS DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1601 E Highway 84, Hayti, MO",31.522888,-90.580611,NaN


Convert to File

In [39]:
fips_MO = fips_merge_drop
#fips_TN = fips_merge_drop

In [40]:
fips_MO.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MO_fips_scraped.csv") 

Review & Compare

In [41]:
fips_MO = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MO_fips_scraped.csv") 
fips_MO

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,498553,498553,2660988802,4/13/21,768,PPP,EASTER SEALS MIDWEST,11933 Westline Industrial Dr,...,999 Other,6232 Residential Intellectual and Developmenta...,6232.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"11933 Westline Industrial Dr, Saint Louis, MO",38.702194,-90.439354,2.918922e+14
1,1,1,498554,498554,3162008008,6/24/20,768,PPP,"4M BUILDING SOLUTIONS, INC.",2827 Clark Avenue,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,ST. LOUIS DISTRICT OFFICE,e $5-10 million,500 or more,"2827 Clark Avenue, SAINT LOUIS, MO",38.629444,-90.220490,2.951013e+14
2,2,2,498555,498555,4904497002,4/4/20,709,PPP,KELLAN RESTAURANT MANAGEMENT CORP,1425 Swift Street Suite 200,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,500 or more,"1425 Swift Street Suite 200, KANSAS CITY, MO",39.134670,-94.577370,2.904702e+14
3,3,3,498556,498556,5467867110,4/13/20,721,PPP,RUSSELL CELLULAR INC,4125 WILSON CREEK MARKETPLACE ROAD,...,999 Other,5179 Other Telecommunications,5179.0,SPRINGFIELD BRANCH OFFICE,e $5-10 million,500 or more,"4125 WILSON CREEK MARKETPLACE ROAD, BATTLEFIEL...",37.140492,-93.367495,2.907700e+14
4,4,4,498557,498557,5529817004,4/5/20,709,PPP,"SUPERIOR BOWEN ASPHALT CO., LLC","520 W. Pennway St., Ste 300",...,324 Petroleum & Coal Products,3241 Petroleum and Coal Products Manufacturing,3241.0,KANSAS CITY DISTRICT OFFICE,e $5-10 million,100 to 249,"520 W. Pennway St., Ste 300, KANSAS CITY, MO",39.087672,-94.591021,2.909502e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16345,16345,16345,514898,514898,9713238502,3/12/21,709,PPS,PREMIER PARKING SERVICES INC.,104 E 5th St Ste 204,...,999 Other,8129 Other Personal Services,8129.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"104 E 5th St Ste 204, Kansas City, MO",39.315845,-94.665410,2.916503e+14
16346,16346,16346,514899,514899,9761878500,3/12/21,709,PPS,J DOOLE PROMOTIONS INC,6130 NW Bell Rd,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"6130 NW Bell Rd, Parkville, MO",39.207040,-94.687126,2.916503e+14
16347,16347,16347,514900,514900,9774858406,2/17/21,709,PPS,BUILDING CONTROL SOLUTIONS LLC,1420 NW Vivion Rd Ste 103,...,999 Other,2382 Building Equipment Contractors,2382.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1420 NW Vivion Rd Ste 103, Kansas City, MO",39.183460,-94.598250,2.904702e+14
16348,16348,16348,514901,514901,9851497109,4/15/20,709,PPP,"EPEC, INC.",5829 TROOST AVE,...,999 Other,8132 Grantmaking and Giving Services,8132.0,KANSAS CITY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5829 TROOST AVE, KANSAS CITY, MO",39.021194,-94.574120,2.909501e+14
